In [25]:
import requests
#import json
#from pyspark.sql import SparkSession, SQLContext, Row 
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark import SparkContext
spark = SparkSession.builder.master("local").appName("north_labs").getOrCreate();
sc = SparkContext("local", "North")



ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=north_labs, master=local) created by getOrCreate at /tmp/ipykernel_212438/3468736556.py:7 

In [2]:
ceps = ['03171060','11065201', '03667000', '00011']
logradouro = []
cidade = []

### função para chamada da API
def consulta_cep():
    
    ### monta dataframe inicial vazio
    #data = ['','','','','','','','','','']
    #deptColumns = ['bairro','cep','complemento','ddd', 'gia','ibge','localidade','logradouro','siafi', 'uf']
    #df_cep = spark.createDataFrame([data], schema=deptColumns)
    #df_cep=df_cep.filter(df_cep.bairro != '')

    contador = 0 
    
    for cep in ceps:
        if len(cep) == 8:
            
            ### efetua a chamada na api para cada cep na lista 
            request = requests.get('https://viacep.com.br/ws/{}/json/'.format(cep))
            cep_data = request.json()
                       
            ### extrai colunas especificas oriundas da API 
            logradouro.append(cep_data['logradouro'] + "," + cep_data['localidade']) 
            cidade.append(cep_data['localidade'])
            
            if contador == 0 :
                df_cep = spark.createDataFrame([cep_data])
                contador = 1 
            else:
                df_temp = spark.createDataFrame([cep_data])
                df_cep = df_cep.union(df_temp)
        
        else: 
            
            print(f'cep errado ! {cep}')

    return df_cep

df = consulta_cep()

df.show()

### cria uma tabela/view a partir do data frame
#df.createOrReplaceTempView('API')

### executa query na tabela/view gerada 
#df_select = spark.sql('select cep from API')
#df_select.show()

#grava csv porem em formato de subpasta com nome incorreto ( na sequencia le o arquivo gerado)
###df_select.coalesce(1).write.format('csv').mode('overwrite').option("header", "true").option("delimiter",",").save('/tmp/novo.txt')
#df.coalesce(1).write.format('csv').mode('overwrite').option("header", "true").option("delimiter",",").save('/tmp/test_northlabs')
#df_read = spark.read.format('csv').option('header', True).load('/tmp/test_northlabs')
#df_read.show()
#df.show()


#grava parquet porem em formato de subpasta com nome incorreto ( na sequencia le o arquivo gerado)
#df.coalesce(1).write.format('parquet').mode('overwrite').option("header", "true").save('/tmp/northlabs_parquet')
#df_read = spark.read.format('parquet').load('/tmp/northlabs_parquet')
#df_read.show()



#grava txt porem em formato de subpasta com nome incorreto ( na sequencia le o arquivo gerado)
#df.coalesce(1).write.format('parquet').mode('overwrite').option("header", "true").save('/tmp/northlabs_parquet')
#df_select = df.select('cep')
#df_select.coalesce(1).write.format('text').mode('overwrite').save('/tmp/test_northlabs_text')
#df_select.coalesce(1).write.format('text').mode('overwrite').option('header', True ).save('/tmp/test_northlabs_text')
#df_read = spark.read.format('text').option('header', True).load('/tmp/test_northlabs_text')
#df_read.show()


#grava somente algumas colunas do dataframe 
#df = df.select('localidade','bairro','cep')
#df.coalesce(1).write.format('parquet').mode('overwrite').option("header", "true").save('/tmp/northlabs_parquet_algumas_colunas')
#df_read = spark.read.format('parquet').load('/tmp/northlabs_parquet_algumas_colunas')
#df_read.show()

#path = '/tmp/arquivo.csv'




### cria uma tabela/view a partir do data frame
#df.createOrReplaceTempView('API')

### ordena cidade e depois mostra somente top 3
#select cidade, qtd from ( select cidade, count(*) as qtd from mawe_silver.clientes group by cidade )  order by 2 desc limit 3
#ou 
#select * from (
#select cidade, qtd, rank() over(order by qtd desc ) as ordenacao from (
#select cidade, count(*) as qtd from mawe_silver.clientes group by cidade ) ) where ordenacao < 4






cep errado ! 00011


+-----------+---------+-----------------+---+----+-------+----------+--------------------+-----+---+
|     bairro|      cep|      complemento|ddd| gia|   ibge|localidade|          logradouro|siafi| uf|
+-----------+---------+-----------------+---+----+-------+----------+--------------------+-----+---+
| Belenzinho|03171-060|                 | 11|1004|3550308| São Paulo|       Rua Paraupava| 7107| SP|
|José Menino|11065-201|de 124/125 ao fim| 13|6336|3548500|    Santos|Avenida President...| 7071| SP|
|    Vila Ré|03667-000|                 | 11|1004|3550308| São Paulo|   Rua Baltazar Brum| 7107| SP|
+-----------+---------+-----------------+---+----+-------+----------+--------------------+-----+---+



In [7]:
df_read = spark.read.format('csv').option('header', True).load('/tmp/test_northlabs')
df_read.show()

+-----------+---------+-----------------+---+----+-------+----------+--------------------+-----+---+
|     bairro|      cep|      complemento|ddd| gia|   ibge|localidade|          logradouro|siafi| uf|
+-----------+---------+-----------------+---+----+-------+----------+--------------------+-----+---+
| Belenzinho|03171-060|             null| 11|1004|3550308| São Paulo|       Rua Paraupava| 7107| SP|
|José Menino|11065-201|de 124/125 ao fim| 13|6336|3548500|    Santos|Avenida President...| 7071| SP|
|    Vila Ré|03667-000|             null| 11|1004|3550308| São Paulo|   Rua Baltazar Brum| 7107| SP|
+-----------+---------+-----------------+---+----+-------+----------+--------------------+-----+---+



In [37]:
#from pyspark.sql import SparkSession
#from pyspark.sql.functions import *
#spark = SparkSession.builder.master("local").appName("north_labs").getOrCreate();

In [ ]:
##df = spark.read.parquet('C:\Saulo\Ambev\Arquivos Parquet\ds_zz_Product')

In [ ]:
#df_parquet = spark.sql('''create database if not exists db_northlabs ''')
#df_parquet = spark.sql('''create table if not exists db_northlabs.InvoiceParquet using parquet location "C:\Saulo\Ambev\ds_zz_Product"''')
#df_parquet = spark.sql('''select distinct InvoiceNumber from db_parquet.InvoiceParquet  where InvoiceNumber <> 0536884625''')
#df_parquet.show()

In [6]:
import requests

auth_token='9251~zv5qY9mCJfdLgJofQMg1YEwhjaa7kPTDTaEOoi4I951wCcdTX6ElGjoLmOP7GjE5'
hed = {'Authorization': 'Bearer ' + auth_token}
#print(hed)
data = '10280'

#url = 'https://acuonline.instructure.com:443/api/v1/users/{}/'.format(data) + 'missing_submissions'
url = 'https://acuonline.instructure.com:443/api/v1/services/kaltura'

#print(url)
response = requests.get(url, headers=hed)
#print(response.status_code)
#print(response.json())

json = response.json()
#print(json)

df = spark.createDataFrame([json])
df.show(truncate=False)


+-----------------------+-------+----------+-----------------------+-----------------------------+
|domain                 |enabled|partner_id|resource_domain        |rtmp_domain                  |
+-----------------------+-------+----------+-----------------------+-----------------------------+
|nv.instructuremedia.com|true   |9         |nv.instructuremedia.com|iad.rtmp.instructuremedia.com|
+-----------------------+-------+----------+-----------------------+-----------------------------+



In [ ]:
https://acuonline.instructure.com:443/api/v1/accounts/1/users?per_page=100

In [3]:
import requests

auth_token='9251~zv5qY9mCJfdLgJofQMg1YEwhjaa7kPTDTaEOoi4I951wCcdTX6ElGjoLmOP7GjE5'
hed = {'Authorization': 'Bearer ' + auth_token}
#print(hed)
data = '10280'

#url = 'https://acuonline.instructure.com:443/api/v1/users/{}/'.format(data) + 'missing_submissions'
url = 'https://acuonline.instructure.com:443/api/v1/accounts/1/users?per_page=100'

#print(url)
response = requests.get(url, headers=hed)
#print(response.status_code)
#print(response.json())

json = response.json()
#print(json)

df = spark.createDataFrame([json])
df.show(truncate=False)

22/09/01 18:17:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------

In [5]:
display(json)

[{'id': 5291,
  'name': 'Elizabeth Aaron',
  'created_at': '2020-03-20T18:26:25-05:00',
  'sortable_name': 'Aaron, Elizabeth',
  'short_name': 'Beth Aaron',
  'sis_user_id': '000585894',
  'integration_id': None,
  'sis_import_id': None,
  'login_id': 'exa20a'},
 {'id': 3807,
  'name': 'Nancy Abbene',
  'created_at': '2019-02-21T01:16:08-06:00',
  'sortable_name': 'Abbene, Nancy',
  'short_name': 'Nancy Abbene',
  'sis_user_id': '000568072',
  'integration_id': None,
  'sis_import_id': None,
  'login_id': 'nxa19b'},
 {'id': 8760,
  'name': 'Lauren Abbott',
  'created_at': '2021-10-08T00:26:37-05:00',
  'sortable_name': 'Abbott, Lauren',
  'short_name': 'Lauren Abbott',
  'sis_user_id': '000602951',
  'integration_id': None,
  'sis_import_id': None,
  'login_id': 'lea21a'},
 {'id': 8135,
  'name': 'Zachary Abbott',
  'created_at': '2021-08-20T18:26:23-05:00',
  'sortable_name': 'Abbott, Zachary',
  'short_name': 'Zachary Abbott',
  'sis_user_id': '000599899',
  'integration_id': None,
 

In [13]:
data_for_pyspark = [{r[0]: r[0] for r in record} for record in json]

spark_df = spark.createDataFrame(data_for_pyspark)

In [14]:
display(spark_df)

DataFrame[c: string, i: string, l: string, n: string, s: string]

In [11]:
spark_df.show()

+---+---+---+---+---+
|  c|  i|  l|  n|  s|
+---+---+---+---+---+
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
|  r|  n|  o|  a|  i|
+---+---+---+---+---+
only showing top 20 rows



In [15]:
json_list = []
json_list.append(json)

df = spark.read.json(sc.parallelize(json_list))
display(df)

NameError: name 'sc' is not defined

In [17]:
import json as j

jsonDataList = []
jsonDataList.append(json)

jsonRDD = spark.parallelize(jsonDataList)
df = spark.read.json(jsonRDD)
display(df)

AttributeError: 'SparkSession' object has no attribute 'parallelize'

In [21]:
results_rdd = north_labs.parallelize([json])
results_df = spark.read.json(results_rdd)
results_df.show()

NameError: name 'north_labs' is not defined

In [20]:
print(spark.sparkContext)
print("Spark App Name : "+ spark.sparkContext.appName)

<SparkContext master=local appName=north_labs>
Spark App Name : north_labs


In [22]:
sc = SparkContext("local", "North")

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=north_labs, master=local) created by getOrCreate at /tmp/ipykernel_212438/3468736556.py:7 